In [13]:
!pip install shortid rdflib PyYAML

In [2]:
import yaml
import glob
import pandas as pd
from lib.utils import translate
from rdflib import plugin, ConjunctiveGraph, URIRef, Literal
from rdflib.store import Store
from rdflib.namespace import RDFS

In [4]:
vocab = dict()
vocab[None] = {'uri': URIRef('http://envri.eu/ns/NULL'), 'label': 'None'}
vocab['NULL'] = {'uri': URIRef('http://envri.eu/ns/NULL'), 'label': 'NULL'}

store = plugin.get('IOMemory', Store)()

g = ConjunctiveGraph(store)
g.bind('envri', 'http://envri.eu/ns/')
g.bind('dcterms', 'http://purl.org/dc/terms/')
g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.bind('skos', 'http://www.w3.org/2004/02/skos/core#')

with open('vocab.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        v = dict()
        v['uri'] = URIRef(value['uri'])
        v['label'] = value['label']
        vocab[key] = v
        g.add((vocab[key]['uri'], RDFS.label, Literal(vocab[key]['label'])))

with open('fairmapping.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        g.add((vocab[key]['uri'], vocab['relatesTo']['uri'], URIRef(value)))
    
for file in glob.glob('descriptions/*.yaml'):
    with open(file, 'r') as f: 
        for document in yaml.load_all(f, Loader=yaml.FullLoader):
            translate(store, document, vocab)
    
g.serialize(destination='data.trig', format='trig')

# OntoWiki does not support named graphs, use this if the data should be loaded in the ENVRI KB
# g.serialize(destination='data.ttl', format='ttl')